In [16]:
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_recall_curve, auc, confusion_matrix, classification_report, recall_score, roc_auc_score, precision_score 
import dalex as dx

# The netherlands - model exploration

In [17]:
netherlands = pd.read_excel('data/42256_2020_253_MOESM1_ESM.xlsx')

In [18]:
df = netherlands[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "Survival/death"]]
df = df.assign(death = np.where(df["Survival/death"] == 'Alive', 0, 1))
df = df[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "death"]]
X = df.drop("death", axis = 1)
y_netherlands = df.death

x_train, x_test, y_train, y_test = train_test_split(X,
                                                    y_netherlands,
                                                    test_size=0.2,
                                                    random_state=1)

In [19]:
model = xgb.XGBClassifier(
            max_depth=4,
            learning_rate=0.2,
            reg_lambda=1,
            n_estimators=150,
            subsample=0.9,
            colsample_bytree=0.9,
            eval_metric = 'aucpr')

In [20]:
numerical_features = ['Age', 'LD', 'CRP', 'Percentage lymphocytes']
numerical_transformer = Pipeline(
    steps=[
        #('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_features = ['Gender']
categorical_transformer = Pipeline(
    steps=[
        #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

clf.fit(x_train, y_train)

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'LD', 'CRP',
                                                   'Percentage lymphocytes']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender'])])),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,...
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interact

In [21]:
y_pred = clf.predict(x_test)
exp = dx.Explainer(clf, x_test, y_test)
exp.model_performance()

Preparation of a new explainer is initiated

  -> data              : 61 rows 5 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 61 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f49a6fed160> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000108, mean = 0.215, max = 0.975
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.971, mean = -0.0346, max = 0.999
  -> model_info        : package sklearn

A new explainer has been created!


,recall,precision,f1,accuracy,auc
XGBClassifier,0.545455,0.461538,0.5,0.803279,0.792727


In [22]:
exp.model_parts().plot()

In [23]:
pdp_num = exp.model_profile(type = 'partial', label="pdp").plot()

Calculating ceteris paribus: 100%|██████████| 5/5 [00:00<00:00, 37.56it/s]


# Should we even test the blood? 

In [24]:
numerical_features = ['Age']
numerical_transformer = Pipeline(
    steps=[('scaler', StandardScaler())])

categorical_features = ['Gender']
categorical_transformer = Pipeline(
    steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

clf_1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

numerical_features = ['LD', 'CRP', 'Percentage lymphocytes']
numerical_transformer = Pipeline(
    steps=[
        #('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),]
)

clf_2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])


In [25]:
age_sex = clf_1.fit(X[['Age','Gender']], y_netherlands)
blood = clf_2.fit(X[['LD', 'CRP', 'Percentage lymphocytes']], y_netherlands)

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



### Precision for age&sex

In [26]:
cross_val_score(age_sex, X[['Age','Gender']], y_netherlands, cv=3, scoring='precision').mean()

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove t

0.40165631469979296

### Precision for blood test

In [27]:
cross_val_score(blood, X[['LD', 'CRP', 'Percentage lymphocytes']], y_netherlands, cv = 3, scoring='precision').mean()

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/hania/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove t

0.26666666666666666

# How does it look in china?

In [28]:
from raport_v1.utils_features_selection import data_read_and_split

In [29]:
china = pd.read_excel('data/time_series_375_prerpocess_en.xlsx')
from raport_v1.utils_features_selection import data_read_and_split
from raport_v1.utils_features_selection import data_read_and_split
X_data_all_features, Y_data, x_col = data_read_and_split()
cheng = {'乳酸脱氢酶':'LDH', '超敏C反应蛋白':'CRP', '淋巴细胞(%)':'Lymphocytes'}
blood_df = X_data_all_features[list(cheng.keys())]
blood_df.columns = list(cheng.values())

age_sex_df = china.loc[china.PATIENT_ID.isin(blood_df.index)][['age','gender','outcome']]
y = age_sex_df.outcome
age_sex_df = age_sex_df.drop('outcome', axis =1)

# default model
age_sex_df_model = model.fit(age_sex_df, y)
blood_model = model.fit(blood_df, y)

### Precision for age&sex

In [30]:
cross_val_score(age_sex_df_model, age_sex_df, y, cv = 3, scoring='precision').mean()

0.7271326079052155

### Precision for blood test

In [31]:
cross_val_score(blood_model, blood_df, y, cv = 3, scoring='precision').mean()

0.9349551414768807

# Confidential Data from NY

In [32]:
ny = pd.read_csv('~/Downloads/Yan_reply_First_last_wtime.csv')

In [33]:
cross_val_score(model, ny[['First_LDH', 'First_CRP', 'First_Lymph']], ny.Expired_Outcome, cv = 3, scoring='precision').mean()

0.4693131871885025

In [34]:
cross_val_score(model, ny[['Last_LDH', 'Last_CRP', 'Last_Lymph']], ny.Expired_Outcome, cv = 3, scoring='precision').mean()

0.7570846039838287

In [38]:
ny

,ClientVisitGUID,Expired_Outcome,Last_LDH,Last_CRP,Last_Lymph,First_LDH,First_CRP,First_Lymph,Last_time_LDH,Last_time_CRP,Last_time_Lymph,First_time_LDH,First_time_CRP,First_time_Lymph
0,1298314042,1,641,57.5,4.5,598,54.30,4.5,2 days 11:12:59,0 days 05:24:53,1 days 21:43:56,2 days 15:51:00,2 days 15:49:10,1 days 21:43:57
1,231729013,1,334,100.4,0.9,334,223.60,0.9,4 days 23:49:43,2 days 23:51:48,6 days 17:33:16,4 days 23:49:43,4 days 22:42:30,35 days 22:23:14
2,292613519,1,742,168.2,2.8,525,16.77,8.1,0 days 03:04:19,0 days 03:04:10,-1 days +23:56:08,14 days 15:58:27,15 days 18:47:49,11 days 00:51:43
3,1624500118,1,613,19.3,0.9,296,19.30,22.1,0 days 08:42:44,2 days 14:31:34,0 days 10:37:16,5 days 00:35:01,2 days 14:31:34,24 days 22:08:05
4,1101470227,1,271,25.1,2.6,374,46.70,5.4,0 days 23:21:10,-1 days +22:54:57,-1 days +20:35:07,24 days 13:05:59,24 days 13:28:36,27 days 01:15:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1945213968,0,301,4.1,8.0,301,126.20,8.0,2 days 05:49:41,2 days 05:54:13,8 days 12:17:24,2 days 05:49:41,8 days 13:37:45,8 days 12:17:24
1034,1468702548,0,1126,69.6,14.2,1126,61.50,15.2,6 days 04:13:47,1 days 06:23:51,2 days 03:52:03,6 days 04:13:47,5 days 23:30:30,6 days 02:58:05
1035,689678610,1,340,39.5,6.1,340,39.50,6.1,1 days 01:07:35,1 days 01:07:35,1 days 06:40:15,1 days 01:07:35,1 days 01:07:35,1 days 06:40:15
1036,431248849,1,491,73.3,21.9,491,73.30,21.9,3 days 05:51:23,3 days 05:56:12,4 days 01:27:16,3 days 05:51:23,3 days 05:56:12,4 days 01:27:16


In [42]:
blood_exp = dx.Explainer(blood_model, blood_df, y)
blood_exp.label = "blood"

arena=dx.Arena()

arena.push_model(age_sex_exp)
arena.push_model(blood_exp)

arena.push_observations(ny)
arena.run_server(port=9294)

Preparation of a new explainer is initiated

  -> data              : 375 rows 3 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 375 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f49a6fed160> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000146, mean = 0.464, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.643, mean = -0.000131, max = 0.357
  -> model_info        : package xgboost

A new explainer has been created!
https://arena.drwhy.ai/?data=http://127.0.0.1:9294/


In [43]:
arena.stop_server()

# All data (apart from france which is bleee) 

In [44]:
china = blood_df
china['death'] = Y_data
china['from'] = 'china'
china = china.reset_index(drop = True)

In [45]:
netherlands = df
y_netherlands = df.death
netherlands = netherlands.iloc[:,2:-2]
netherlands['death'] = y_netherlands
netherlands['from'] = 'netherlands'
netherlands = netherlands.rename(columns = { 'Percentage lymphocytes':'Lymphocytes', 'LD':'LDH'})

In [46]:
y_ny = ny.Expired_Outcome
newyork = ny[['First_LDH', 'First_CRP', 'First_Lymph']]
newyork = newyork.rename(columns = {'First_LDH':'LDH',
                                      'First_CRP':'CRP',
                                      'First_Lymph':'Lymphocytes'})
newyork['from'] = 'newyork'
newyork['death'] = y_ny

In [47]:
nations = china.append([newyork, netherlands])
nations['death'] = nations.death.astype('int')
nations = nations.reset_index(drop=True)


In [48]:
X = nations[['LDH','CRP','Lymphocytes','from']]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, nations.death, test_size = 0.33, random_state = 42)

In [50]:
model.fit(X_train.iloc[:,:-1], y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, eval_metric='aucpr',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=150, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.9, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [51]:
exp = dx.Explainer(model, X_test.iloc[:,:-1], y_test)

Preparation of a new explainer is initiated

  -> data              : 567 rows 3 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 567 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f49a6fed160> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000409, mean = 0.342, max = 0.993
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.906, mean = 0.0215, max = 0.987
  -> model_info        : package xgboost

A new explainer has been created!


In [52]:
exp.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.514563,0.619883,0.562334,0.708995,0.757026


In [54]:
arena=dx.Arena()
arena.push_model(exp)

arena.push_observations(X_test)
arena.run_server(port=9295)

https://arena.drwhy.ai/?data=http://127.0.0.1:9295/


# Fairness

In [48]:
exp.model_fairness(protected = X_test['from'], privileged = 'china').plot(type='fairness_check')

In [49]:
exp.model_fairness(protected = X_test['from'], privileged = 'china').plot(type='metric_scores')

In [51]:
exp.model_fairness(protected = X_test['from'], privileged = 'china').plot(type='radar')